<a href="https://colab.research.google.com/github/iaxolot/SteamlitS/blob/main/agentLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
import streamlit as st
from datetime import datetime
import os
import html

search_tool = SerperDevTool()

os.environ["OPENAI_API_BASE"] = 'https://api.groq.com/openai/v1'
os.environ["OPENAI_MODEL_NAME"] = 'Llama3-70b-8192' # 'llama-3.1-70b-versatile'  # Adjust based on available model
os.environ["OPENAI_API_KEY"] = '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'
os.environ["SERPER_API_KEY"] = '1ef43478a3ea96aa18e4912dbb9e37f03c2696d6'

import time
from typing import List
import queue
import threading
import io
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

from fpdf import FPDF, HTMLMixin
import markdown2

# Configuración de la página
st.set_page_config(
    page_title="Análisis de Tendencias",
    page_icon="📊",
    layout="wide"
)

# Título y descripción
st.title("📊 Análisis de Tendencias y Políticas")
st.markdown("""
<style>
    .stApp {
        background-image: url("https://wallpapercave.com/wp/wp12651903.jpg");
        background-size: center;
        background-position: center;
    }
    </style>

Esta aplicación analiza las últimas tendencias en el tema de tu elección utilizando agentes de IA.
El análisis incluye búsqueda de información actualizada y generación de resúmenes.
""",
    unsafe_allow_html=True)

# Crear una cola para los mensajes de log
if 'log_queue' not in st.session_state:
    st.session_state.log_queue = queue.Queue()

# Crear una lista para almacenar los mensajes de log
if 'log_messages' not in st.session_state:
    st.session_state.log_messages = []

class VerboseCallback:
    def __init__(self, queue):
        self.queue = queue

    def __call__(self, message):
        # Agregar timestamp al mensaje
        timestamp = datetime.now().strftime("%H:%M:%S")
        formatted_message = f"[{timestamp}] {message}"
        self.queue.put(formatted_message)

with st.sidebar:
    st.header("Configuración")

    # Campo para el tema de análisis
    topic = st.text_input(
        "Tema a analizar",
        value="Industrial Policy",
        help="Ingresa el tema sobre el cual quieres realizar el análisis"
    )

    # Opciones adicionales
    language = st.selectbox(
        "Idioma del reporte",
        options=["Español", "English"],
        index=0,
        help="Selecciona el idioma en el que deseas recibir el análisis"
    )

    st.markdown("---")
    st.markdown("""
    ### Sobre esta aplicación
    Esta herramienta utiliza agentes de IA para:
    - Investigar tendencias actuales
    - Analizar avances y desarrollos
    - Generar resúmenes coherentes
    """)

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        # Establecer fuente DejaVu que soporta Unicode
        self.add_font('DejaVu', '', 'DejaVuSansCondensed.ttf', uni=True)
        self.add_font('DejaVu', 'B', 'DejaVuSansCondensed-Bold.ttf', uni=True)

def save_result_as_pdf_with_markdown(resultado, topic, language):
    html_content = markdown2.markdown(resultado)
    buffer = io.BytesIO()
    pdf = SimpleDocTemplate(buffer, pagesize=A4)

    styles = getSampleStyleSheet()
    title_style = ParagraphStyle('TitleStyle', parent=styles['Title'], fontSize=16, alignment=1)
    content_style = ParagraphStyle('ContentStyle', parent=styles['BodyText'], fontSize=12, leading=14)

    elements = []
    title = f"Análisis de {topic}" if language == "Español" else f"Analysis of {topic}"
    elements.append(Paragraph(title, title_style))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(html_content, content_style))
    pdf.build(elements)

    buffer.seek(0)
    pdf_data = buffer.getvalue()
    buffer.close()
    return pdf_data

def save_result_as_text(resultado, topic):
    if not os.path.exists('resultados'):
        os.makedirs('resultados')

    ahora = datetime.now()
    nombre_archivo = f"resultados/resultado_{topic.replace(' ', '_')}_{ahora.strftime('%Y-%m-%d_%H-%M-%S')}.txt"

    with open(nombre_archivo, 'w', encoding='utf-8') as archivo:
        archivo.write(str(resultado))

    return nombre_archivo

class ProgressCallback:
    def __init__(self, progress_bar, status_text):
        self.progress_bar = progress_bar
        self.status_text = status_text
        self.current_step = 0
        self.total_steps = 5

    def update(self, description):
        self.current_step += 1
        progress = min(self.current_step / self.total_steps, 1.0)
        self.progress_bar.progress(progress)
        self.status_text.text(f"Estado: {description}")
        time.sleep(0.1)

# Crear un layout de dos columnas
col1, col2 = st.columns([2, 1])

with col1:
    # Área principal para resultados
    if len(topic.strip()) < 3:
        st.warning("Por favor, ingresa un tema con al menos 3 caracteres.")
    else:
        st.markdown(f"### Tema seleccionado: **{topic}**")

        # Botón de inicio
        if st.button("Iniciar Análisis", type="primary"):
            # Limpiar logs anteriores
            st.session_state.log_messages = []

            # Crear áreas para la barra de progreso
            progress_bar = st.progress(0)
            status_text = st.empty()

            # Crear callbacks
            progress_callback = ProgressCallback(progress_bar, status_text)
            verbose_callback = VerboseCallback(st.session_state.log_queue)

            try:
                # Configuración inicial
                progress_callback.update(f"Configurando agente de investigación para {topic}...")
                verbose_callback(f"Iniciando análisis de {topic}")

                # Crear agentes
                researcher = Agent(
                    role='Researcher',
                    goal=f'Look for information about {topic} news and developments',
                    backstory=f"""You work at a leading think tank specialized in {topic}.
                        Your expertise lies in searching and analyzing information about {topic}.
                        You have a knack for dissecting complex data and presenting actionable insights.""",
                    tools=[search_tool],
                    verbose=True,
                    allow_delegations=False
                )

                progress_callback.update("Preparando agente escritor...")
                verbose_callback("Configurando agente de redacción")

                writer = Agent(
                    role='Professional Content Writer',
                    goal=f'summarize the latest advancement in {topic} in a concise article and redact in markdown format. Always write the article in {"spanish" if language == "Español" else "english"}',
                    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
                    You transform complex concepts into compelling narratives.""",
                    verbose=True,
                    allow_delegation=True
                )

                progress_callback.update("Definiendo tareas de análisis...")
                verbose_callback("Definiendo tareas para los agentes")

                # Crear tareas
                tarea1 = Task(
                    description=f"""Conduct a comprehensive analysis of the latest advancements in {topic} for year 2024.
                    Identify key trends, breakthrough developments, and potential impacts.""",
                    expected_output="Full analysis report in bullet points",
                    agent=researcher
                )

                tarea2 = Task(
                    description=f"""Review the comprehensive analysis from Task 1 and curate a list of the best references
                    and online links to further analyze the topic in-depth.
                    The output should include sources that are reliable, and insightful, and cover different perspectives on {topic}.""",
                    expected_output="A detailed list of links and references for further research",
                    agent=researcher  # Reutilizando el agente researcher
                )

                tarea3 = Task(
                    description=f"""Using the insights provided in Task 1, write a short article
                    that highlights the most significant {topic} advancements.
                    Your post should be informative yet accessible, catering to a specialist audience.
                    Make it sound engaging, avoid complex words so it doesn't sound like AI.
                    Include the references curated in Task 2 at the end of the article.
                    Write the article in {"spanish" if language == "Español" else "english"}.""",
                    expected_output="Full blog post of at least 5 paragraphs",
                    agent=writer
                )

                progress_callback.update(f"Ejecutando análisis de {topic}...")
                verbose_callback("Iniciando proceso de análisis")

                # Crear y ejecutar crew
                crew = Crew(
                    agents=[researcher, writer],
                    tasks=[tarea1, tarea2, tarea3],
                    verbose=True
                )

                # Ejecutar el análisis
                resultado = crew.kickoff()
                resultado_texto = str(resultado)

                progress_callback.update("Generando reporte...")
                verbose_callback("Generando reporte del análisis")

                # Generar PDF
                try:
                    pdf_data = save_result_as_pdf_with_markdown(resultado_texto, topic, language)
                    progress_callback.update("Guardando resultados...")
                    verbose_callback("Guardando resultados del análisis")

                    # Guardar resultado como texto
                    archivo_guardado = save_result_as_text(resultado_texto, topic)
                    progress_callback.update("¡Análisis completado!")
                    verbose_callback("Proceso completado exitosamente")

                    # Mostrar resultados en tabs
                    tab1, tab2, tab3 = st.tabs(["Resultado", "Archivo PDF", "Archivo Texto"])

                    with tab1:
                        st.markdown(f"### Análisis de {topic}")
                        st.markdown(resultado_texto)

                    with tab2:
                        st.markdown("### Reporte en PDF")
                        if pdf_data:  # Verificar que tenemos datos del PDF
                            st.download_button(
                                label="Descargar Reporte PDF",
                                data=pdf_data,
                                file_name=f"reporte_{topic.replace(' ', '_')}.pdf",
                                mime="application/pdf"
                            )
                        else:
                            st.error("Error al generar el PDF")

                    with tab3:
                        st.markdown("### Archivo de Texto")
                        st.info(f"Resultado guardado en: {archivo_guardado}")

                        with open(archivo_guardado, 'r', encoding='utf-8') as file:
                            st.download_button(
                                label="Descargar Resultado TXT",
                                data=file,
                                file_name=os.path.basename(archivo_guardado),
                                mime="text/plain"
                            )

                except Exception as e:
                    st.error(f"Error al generar el PDF: {str(e)}")
                    verbose_callback(f"Error: {str(e)}")

            except Exception as e:
                progress_callback.update("Error en el análisis")
                verbose_callback(f"Error: {str(e)}")
                st.error(f"Ocurrió un error durante el análisis: {str(e)}")

with col2:
    # Área para el log en tiempo real
    st.markdown("### Log de Actividades")
    log_container = st.empty()

    # Procesar mensajes en la cola
    while not st.session_state.log_queue.empty():
        message = st.session_state.log_queue.get()
        st.session_state.log_messages.append(message)

    # Mostrar todos los mensajes en el contenedor
    if st.session_state.log_messages:
        log_text = "\n".join(st.session_state.log_messages)
        log_container.code(log_text, language="text")
    else:
        log_container.markdown("*Esperando inicio del análisis...*")